## For EDA to keep thing short I will use SQL to get answers for few Questions Only

In [5]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

In [2]:
%%capture
%run ../eda/read_data.ipynb

In [4]:
import sqlite3

conn = sqlite3.connect(":memory:")

users.to_sql('users', conn, index=False)
train_df.to_sql('content', conn, index=False)
events.to_sql('events', conn, index=False)

3544161

Q1. How many unique users do we have?

In [15]:
query = """
SELECT COUNT(DISTINCT deviceId) AS active_users FROM users
"""
print(pd.read_sql(query, conn))

   active_users
0         10400


Q2. How many unique contents are available in training data?

In [7]:
query = """
SELECT COUNT(DISTINCT hashid) AS unique_contents
FROM Content
WHERE createdAt IS NOT NULL
"""
print(pd.read_sql(query, conn))

   unique_contents
0             8153


Q3. How many user–content interactions (training samples) do we have?

In [10]:
query = """
SELECT COUNT(*) AS total_interactions
FROM events
WHERE hashId IS NOT NULL
"""
print(pd.read_sql(query, conn))

   total_interactions
0             3544154


Q4. How many unique user–content pairs are there? (to remove duplicates)

In [11]:
query = """
SELECT COUNT(DISTINCT CONCAT(deviceId, '_', hashId)) AS unique_user_content_pairs
FROM Events
"""
print(pd.read_sql(query, conn))

   unique_user_content_pairs
0                    2766174


Q5. How many distinct event types are there, and what’s their distribution?

In [14]:
query = """
SELECT event_type, COUNT(*) AS count, 
       ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM Events
GROUP BY event_type
ORDER BY count DESC
"""
print(pd.read_sql(query, conn))

                  event_type    count  percentage
0            TimeSpent-Front  3480131       98.19
1             TimeSpent-Back    44933        1.27
2            News Bookmarked    10870        0.31
3                News Shared     3517        0.10
4          News Unbookmarked     2275        0.06
5  Relevancy Option Selected     1312        0.04
6                     Search     1123        0.03


Q6. How many users have at least one event?

In [16]:
query = """
SELECT COUNT(DISTINCT deviceId) AS active_users
FROM Events
"""
print(pd.read_sql(query, conn))

   active_users
0          8977


Q7. How recent is the training data?

In [18]:
query = """
SELECT 
    MIN(eventTimestamp) AS first_event,
    MAX(eventTimestamp) AS last_event
FROM Events
"""
print(pd.read_sql(query, conn))

     first_event     last_event
0  1687824007000  1690329599000


Q8. Which users are most/least active? (outliers)

In [20]:
query = """
SELECT deviceId, COUNT(*) AS num_events
FROM Events
GROUP BY deviceId
ORDER BY num_events DESC
LIMIT 10
"""
print(pd.read_sql(query, conn))

                               deviceId  num_events
0  c8f40125-55ee-4b6b-82a9-a60fc8a5017a       21363
1  3075977c-5143-4abb-86df-180c9468b488       19392
2  3715b58b-04f5-4093-bbef-1169d9cd3081       15103
3  68738cd7-ae73-49c7-90d0-9829516f434e       14664
4  d5f65726-1e26-465c-899a-6e1c95507392       14018
5  6c16db9b-b0d1-4151-8a99-50f991dd5054       13912
6  b2e6a95d-eed8-43dc-bec7-6bceac448c50       13884
7  049b40db-bfda-4f61-9118-708e831e9823       13167
8  459aa26e-3799-44fc-8391-26efc668c357       11721
9  7493ab4f-8f7c-446d-a095-5f60b5cfca66       11718


Q9. What is the average and extreme overallTimeSpent per event type?

In [22]:
query = """
SELECT event_type,
       AVG(CAST(overallTimeSpent AS FLOAT)) AS avg_time_spent,
       MAX(CAST(overallTimeSpent AS FLOAT)) AS max_time_spent
FROM Events
WHERE overallTimeSpent IS NOT NULL
GROUP BY event_type
ORDER BY avg_time_spent DESC
"""
print(pd.read_sql(query, conn))

                  event_type  avg_time_spent  max_time_spent
0             TimeSpent-Back       41.184386        3744.791
1            TimeSpent-Front        8.592099       41047.590
2  Relevancy Option Selected        0.000000           0.000
3          News Unbookmarked        0.000000           0.000
4                News Shared        0.000000           0.000
5            News Bookmarked        0.000000           0.000


Q10. How many contents or users have zero events (cold-start cases)?

In [27]:
# Users with no events
query = """
SELECT COUNT(*) AS cold_start_users
FROM Users u
LEFT JOIN Events e ON u.deviceid = e.deviceId
WHERE e.deviceId IS NULL
"""
print(pd.read_sql(query, conn))

# Contents with no engagement
query = """
SELECT COUNT(*) AS cold_start_contents
FROM Content c
LEFT JOIN Events e ON c.hashid = e.hashId
WHERE e.hashId IS NULL
"""
print(pd.read_sql(query, conn))

   cold_start_users
0              9690
   cold_start_contents
0                   15


Q11. Which categories or languages dominate the content? (imbalance)

In [28]:
query = """
SELECT categories, COUNT(*) AS count
FROM Content
GROUP BY categories
ORDER BY count DESC
LIMIT 10
"""
print(pd.read_sql(query, conn))


query = """
SELECT newsLanguage, COUNT(*) AS count
FROM Content
GROUP BY newsLanguage
ORDER BY count DESC
"""
print(pd.read_sql(query, conn))

          categories  count
0           national   2640
1              world   1006
2             sports    745
3           business    536
4      entertainment    465
5  politics,national    387
6           politics    193
7     world,national    185
8         technology    181
9         VIDEO_NEWS    165
  newsLanguage  count
0      english   6211
1        hindi   1845
2         None    109
3     gujarati      3
4      Twitter      1
5          ANI      1


Q12. How much data will I have for training (after cleaning)?

In [30]:
query = """
SELECT COUNT(*) AS usable_samples
FROM Events
WHERE hashId IS NOT NULL 
  AND deviceId IS NOT NULL 
  AND overallTimeSpent IS NOT NULL 
  AND CAST(overallTimeSpent AS FLOAT) BETWEEN 1 AND 600
"""
print(pd.read_sql(query, conn))

   usable_samples
0         2812221


Q13. Are there any missing or invalid timestamps?

In [32]:
query = """
SELECT 
    SUM(CASE WHEN eventTimestamp IS NULL THEN 1 ELSE 0 END) AS missing_timestamps,
    SUM(CASE WHEN eventTimestamp < 0 THEN 1 ELSE 0 END) AS invalid_timestamps
FROM Events
"""
print(pd.read_sql(query, conn))

   missing_timestamps  invalid_timestamps
0                   0                   0


Q14. Any abnormal user behavior (too many unique contents per user)?

In [34]:
query = """
SELECT deviceId, COUNT(DISTINCT hashId) AS unique_contents
FROM Events
GROUP BY deviceId
ORDER BY unique_contents DESC
LIMIT 10
"""
print(pd.read_sql(query, conn))

                               deviceId  unique_contents
0  459aa26e-3799-44fc-8391-26efc668c357             8130
1  c8f40125-55ee-4b6b-82a9-a60fc8a5017a             8110
2  68738cd7-ae73-49c7-90d0-9829516f434e             8106
3  3075977c-5143-4abb-86df-180c9468b488             8088
4  de04f7a9-db93-4aed-93ee-98c2d2d0c135             8080
5  28657cbf-680e-4475-b281-cb585229a569             8041
6  eb22d2b1-d67a-48e4-afcd-fa1708c833ca             8029
7  65c92ded-26bd-4594-a5af-97b13feef4ec             8029
8  023845e6-4ae8-4de3-b83a-6d17e846b17d             8029
9  c9718252-757f-4b65-8e4d-4dcb031c0b6f             8012


Q15. Are there duplicate content IDs or reused hashids?

In [38]:
query = """
SELECT hashid, COUNT(*) AS count
FROM Content
GROUP BY hashid
HAVING COUNT(*) > 1
"""
print(pd.read_sql(query, conn))

                                              hashid  count
0  Twitter now limits users on how many tweets th...      2
1           invite Sarma to his place in New Delhi."      2


# strange hashid's since broken CSV